# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from fastai.data.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *
import cv2

In [ ]:
from tqdm.notebook import tqdm
from sklearn import metrics

# Load Data

In [ ]:
train_targets = pd.read_csv("../input/garage-detection-unofficial-ssl-challenge/image_labels_train.csv")
holdout_targets = pd.read_csv("../input/garage-detection-unofficial-ssl-challenge/image_labels_holdout.csv")

In [ ]:
train_targets.head()

create new directories

In [ ]:
!mkdir ./garage/
!mkdir ./other/
!ls

move images to corresponding directories

In [ ]:
def move_image(path1, path2):
    # creating a image object (main image)
    img = cv2.imread(path1) 
    # save a image using extension 
    cv2.imwrite(path2, img) 

In [ ]:
%%time

for index, row in train_targets.iterrows():
    if row['GarageDoorEntranceIndicator']==1:
        move_image(f"../input/garage-detection-unofficial-ssl-challenge/GarageImages/GarageImages/image{row['ID']}.jpg", 
                   f"./garage/image{row['ID']}.jpg")
    else:
        move_image(f"../input/garage-detection-unofficial-ssl-challenge/GarageImages/GarageImages/image{row['ID']}.jpg", 
                   f"./other/image{row['ID']}.jpg")
        
print("Total garage images: ", len(os.listdir("./garage/")))
print("Total non-garage images: ", len(os.listdir("./other/")))

create DataBlock

In [ ]:
garages = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

garages

verify images

In [ ]:
fns = get_image_files('./')
fns

In [ ]:
failed = verify_images(fns)
failed

create dataloader and show a batch

In [ ]:
path = Path('./')
path

In [ ]:
dls = garages.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

show batch with squish transformation

In [ ]:
garages = garages.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = garages.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

show batch with pad transformation

In [ ]:
garages = garages.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = garages.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

show batch with re-crop transformation

In [ ]:
garages = garages.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = garages.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
garages = garages.new(item_tfms=RandomResizedCrop(128, min_scale=0.5))
dls = garages.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

show some augmentation

In [ ]:
garages = garages.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = garages.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

# Training

In [ ]:
garages = garages.new(
    item_tfms=RandomResizedCrop(128, min_scale=0.5),
    batch_tfms=aug_transforms()
)
dls = garages.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

# Inference

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
holdout_targets = pd.read_csv("../input/garage-detection-unofficial-ssl-challenge/image_labels_holdout.csv")
holdout_targets.head()

In [ ]:
learn_inf.dls.vocab

In [ ]:
y_pred = []
y_true = []
for index, row in tqdm(holdout_targets.iterrows()):
    im_path = "../input/garage-detection-unofficial-ssl-challenge/GarageImages/GarageImages"
    im_file_path = im_path + "/image" + str(row['ID']) + ".jpg"
    y_pred.append(learn_inf.predict(im_file_path)[2].numpy()[0])
    y_true.append(row['GarageDoorEntranceIndicator'])

# Score

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=1)
print("final holdout auc: ", metrics.auc(fpr, tpr))